# Trading Strategy Visualization

This notebook focuses on creating advanced visualizations for our trading model results, including:
- Interactive price charts with entry points
- Technical indicator visualizations
- Performance metrics visualization
- Trade result analysis charts

In [2]:
# Add parent directory to path to import from src
import sys
import os
sys.path.append(os.path.abspath('..'))

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Import from src modules
from src.data.loader import load_data, preprocess_data
from src.data.features import prepare_features
from src.models.random_forest_model import RandomForestModel
from src.visualization.charts import plot_technical_indicators, plot_backtest_results
from src.utils.helpers import set_pandas_display_options

# Set display options
set_pandas_display_options()

# Matplotlib settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6]
%matplotlib inline

## 1. Load Data and Results

First, let's load our data and any saved results from previous notebooks.

In [3]:
# Load price data
try:
    # Try to load processed data
    df_features = pd.read_csv('../processed_data.csv', index_col=0, parse_dates=True)
    print(f"Loaded processed dataset with {df_features.shape[1]} columns and {df_features.shape[0]} rows")
except FileNotFoundError:
    print("Processed data file not found. Processing raw data...")
    # Load raw data
    file_path = '../USATECH.IDXUSD_Candlestick_15_M_BID_01.01.2023-18.01.2025.csv'
    df_raw = load_data(file_path)
    df = preprocess_data(df_raw)
    
    # Prepare features
    df_features = prepare_features(df)
    print(f"Prepared dataset with {df_features.shape[1]} columns and {df_features.shape[0]} rows")

Loaded processed dataset with 37 columns and 49815 rows


In [4]:
# Try to load backtest results
try:
    backtest_results = pd.read_csv('../backtest_results.csv', parse_dates=['timestamp', 'exit_timestamp'])
    print(f"Loaded backtest results with {len(backtest_results)} trades")
except FileNotFoundError:
    backtest_results = None
    print("Backtest results file not found. Some visualizations will be skipped.")

Backtest results file not found. Some visualizations will be skipped.


## 2. Interactive Price Chart with Technical Indicators

Let's create an interactive price chart with technical indicators using Plotly.

In [ ]:
# Select a subset of data for visualization
# Use recent 100 periods
recent_data = df_features.iloc[-100:]

# Check technical indicator columns
technical_indicators = []
if 'SMA20' in df_features.columns:
    technical_indicators.append('SMA20')
if 'SMA50' in df_features.columns:
    technical_indicators.append('SMA50')
if 'SMA200' in df_features.columns:
    technical_indicators.append('SMA200')
if 'BB_Upper' in df_features.columns and 'BB_Lower' in df_features.columns:
    technical_indicators.extend(['BB_Upper', 'BB_Lower'])

# Create interactive candlestick chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.1, 
                    row_heights=[0.7, 0.3],
                    subplot_titles=('Price Chart', 'Volume'))

# Add candlestick chart
fig.add_trace(
    go.Candlestick(
        x=recent_data.index,
        open=recent_data['Open'],
        high=recent_data['High'],
        low=recent_data['Low'],
        close=recent_data['Close'],
        name='Price'
    ),
    row=1, col=1
)

# Add technical indicators
for indicator in technical_indicators:
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=recent_data[indicator],
            name=indicator,
            line=dict(width=1)
        ),
        row=1, col=1
    )

# Add volume bar chart
fig.add_trace(
    go.Bar(
        x=recent_data.index,
        y=recent_data['Volume'],
        name='Volume',
        marker_color='rgba(0, 0, 255, 0.5)'
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title='USATECH Index Interactive Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    height=800,
    width=1000,
    legend=dict(orientation='h', y=1.02),
    xaxis_rangeslider_visible=False
)

fig.show()

## 3. Interactive Technical Indicator Dashboard

Let's create an interactive dashboard showing key technical indicators.

In [ ]:
# Check if we have all required indicators
required_indicators = ['RSI', 'MACD', 'MACD_Signal', 'ATR']
have_indicators = all(indicator in df_features.columns for indicator in required_indicators)

if have_indicators:
    # Create a multi-panel chart with technical indicators
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                      vertical_spacing=0.05, 
                      row_heights=[0.4, 0.2, 0.2, 0.2],
                      subplot_titles=('Price', 'RSI', 'MACD', 'ATR'))
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=recent_data.index,
            open=recent_data['Open'],
            high=recent_data['High'],
            low=recent_data['Low'],
            close=recent_data['Close'],
            name='Price'
        ),
        row=1, col=1
    )
    
    # Add moving averages
    for indicator in technical_indicators:
        fig.add_trace(
            go.Scatter(
                x=recent_data.index,
                y=recent_data[indicator],
                name=indicator,
                line=dict(width=1)
            ),
            row=1, col=1
        )
    
    # Add RSI
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=recent_data['RSI'],
            name='RSI',
            line=dict(color='purple', width=1)
        ),
        row=2, col=1
    )
    
    # Add RSI overbought/oversold lines
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=[70] * len(recent_data),
            name='Overbought (70)',
            line=dict(color='red', width=1, dash='dash')
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=[30] * len(recent_data),
            name='Oversold (30)',
            line=dict(color='green', width=1, dash='dash')
        ),
        row=2, col=1
    )
    
    # Add MACD
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=recent_data['MACD'],
            name='MACD',
            line=dict(color='blue', width=1)
        ),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=recent_data['MACD_Signal'],
            name='MACD Signal',
            line=dict(color='red', width=1)
        ),
        row=3, col=1
    )
    
    # Add MACD histogram if available
    if 'MACD_Hist' in df_features.columns:
        fig.add_trace(
            go.Bar(
                x=recent_data.index,
                y=recent_data['MACD_Hist'],
                name='MACD Histogram',
                marker_color=['green' if val >= 0 else 'red' for val in recent_data['MACD_Hist']]
            ),
            row=3, col=1
        )
    
    # Add ATR
    fig.add_trace(
        go.Scatter(
            x=recent_data.index,
            y=recent_data['ATR'],
            name='ATR',
            line=dict(color='orange', width=1)
        ),
        row=4, col=1
    )
    
    # Update layout
    fig.update_layout(
        title='Technical Indicator Dashboard',
        height=900,
        width=1000,
        legend=dict(orientation='h', y=1.02),
        xaxis_rangeslider_visible=False
    )
    
    # Update y-axis ranges
    fig.update_yaxes(title_text='Price', row=1, col=1)
    fig.update_yaxes(title_text='RSI', range=[0, 100], row=2, col=1)
    fig.update_yaxes(title_text='MACD', row=3, col=1)
    fig.update_yaxes(title_text='ATR', row=4, col=1)
    
    fig.show()
else:
    print("Some required technical indicators are missing in the dataset. Skipping this visualization.")

## 4. Visualize Entry Points and Trade Results

If we have backtest results, let's create an interactive visualization of our trades.

In [ ]:
if backtest_results is not None:
    # Create a visualization showing entry and exit points
    # First, merge our backtest results with the price data
    
    # Create a function to get data ranges
    def get_data_range(timestamp, window=50):
        """Get a data range centered around a timestamp"""
        if isinstance(timestamp, str):
            timestamp = pd.to_datetime(timestamp)
        
        # Find the index closest to the timestamp
        idx = df_features.index.get_indexer([timestamp], method='nearest')[0]
        
        # Get window periods before and after
        start_idx = max(0, idx - window//2)
        end_idx = min(len(df_features) - 1, idx + window//2)
        
        return df_features.iloc[start_idx:end_idx+1]
    
    # Create visualizations for each trade or select a subset
    # Let's visualize the first 5 trades
    trades_to_visualize = min(5, len(backtest_results))
    
    for i in range(trades_to_visualize):
        trade = backtest_results.iloc[i]
        
        # Get data range around the trade
        trade_data = get_data_range(trade['timestamp'], window=50)
        
        # Create candlestick chart
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                          vertical_spacing=0.1, 
                          row_heights=[0.7, 0.3],
                          subplot_titles=('Price Chart', 'Volume'))
        
        fig.add_trace(
            go.Candlestick(
                x=trade_data.index,
                open=trade_data['Open'],
                high=trade_data['High'],
                low=trade_data['Low'],
                close=trade_data['Close'],
                name='Price'
            ),
            row=1, col=1
        )
        
        # Add moving averages
        for indicator in technical_indicators:
            fig.add_trace(
                go.Scatter(
                    x=trade_data.index,
                    y=trade_data[indicator],
                    name=indicator,
                    line=dict(width=1)
                ),
                row=1, col=1
            )
        
        # Add volume
        fig.add_trace(
            go.Bar(
                x=trade_data.index,
                y=trade_data['Volume'],
                name='Volume'
            ),
            row=2, col=1
        )
        
        # Mark entry point
        entry_color = 'green' if trade['direction'] == 'LONG' else 'red'
        fig.add_trace(
            go.Scatter(
                x=[trade['timestamp']],
                y=[trade['entry_price']],
                mode='markers',
                marker=dict(size=12, color=entry_color, symbol='triangle-up' if trade['direction'] == 'LONG' else 'triangle-down'),
                name=f"{trade['direction']} Entry"
            ),
            row=1, col=1
        )
        
        # Mark exit point
        exit_color = 'green' if trade['profit_pct'] > 0 else 'red'
        fig.add_trace(
            go.Scatter(
                x=[trade['exit_timestamp']],
                y=[trade['exit_price']],
                mode='markers',
                marker=dict(size=12, color=exit_color, symbol='circle'),
                name='Exit'
            ),
            row=1, col=1
        )
        
        # Mark target price
        fig.add_trace(
            go.Scatter(
                x=[trade['timestamp']],
                y=[trade['target_price']],
                mode='markers',
                marker=dict(size=8, color='green', symbol='star'),
                name='Target'
            ),
            row=1, col=1
        )
        
        # Mark stop loss
        fig.add_trace(
            go.Scatter(
                x=[trade['timestamp']],
                y=[trade['stop_loss_price']],
                mode='markers',
                marker=dict(size=8, color='red', symbol='x'),
                name='Stop Loss'
            ),
            row=1, col=1
        )
        
        # Add horizontal lines for target and stop
        fig.add_trace(
            go.Scatter(
                x=[trade['timestamp'], trade['exit_timestamp']],
                y=[trade['target_price'], trade['target_price']],
                mode='lines',
                line=dict(color='green', width=1, dash='dash'),
                name='Target Line'
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=[trade['timestamp'], trade['exit_timestamp']],
                y=[trade['stop_loss_price'], trade['stop_loss_price']],
                mode='lines',
                line=dict(color='red', width=1, dash='dash'),
                name='Stop Loss Line'
            ),
            row=1, col=1
        )
        
        # Update layout
        outcome_color = 'green' if trade['outcome'] == 'WIN' else 'red'
        title_text = (f"Trade {i+1}: {trade['direction']} - {trade['outcome']} - "
                     f"Profit: {trade['profit_pct']:.2f}% - "
                     f"Held: {trade['bars_held']} bars")
        
        fig.update_layout(
            title=dict(
                text=title_text,
                font=dict(color=outcome_color)
            ),
            height=700,
            width=1000,
            legend=dict(orientation='h', y=1.02),
            xaxis_rangeslider_visible=False
        )
        
        fig.show()
else:
    print("No backtest results available. Skipping trade visualization.")

## 5. Performance Metrics Dashboard

Let's create a dashboard of key performance metrics from our backtest.

In [ ]:
if backtest_results is not None:
    # Create a dashboard of key performance metrics
    
    # 1. Win rate pie chart
    win_count = len(backtest_results[backtest_results['outcome'] == 'WIN'])
    loss_count = len(backtest_results[backtest_results['outcome'] == 'LOSS'])
    incomplete_count = len(backtest_results[backtest_results['outcome'] == 'INCOMPLETE'])
    
    win_rate = win_count / len(backtest_results) * 100
    loss_rate = loss_count / len(backtest_results) * 100
    incomplete_rate = incomplete_count / len(backtest_results) * 100
    
    labels = ['Win', 'Loss', 'Incomplete']
    values = [win_rate, loss_rate, incomplete_rate]
    colors = ['green', 'red', 'gray']
    
    fig = go.Figure(
        data=[go.Pie(
            labels=labels,
            values=values,
            marker=dict(colors=colors),
            textinfo='label+percent',
            hoverinfo='label+percent+value',
            hole=0.4
        )]
    )
    
    fig.update_layout(
        title='Trade Outcome Distribution',
        annotations=[dict(text=f"{win_rate:.1f}% Win Rate", x=0.5, y=0.5, font_size=20, showarrow=False)]
    )
    
    fig.show()
    
    # 2. Profit/Loss distribution
    fig = px.histogram(
        backtest_results, 
        x='profit_pct', 
        nbins=20, 
        color_discrete_sequence=['blue'],
        opacity=0.7,
        marginal='box'
    )
    
    fig.update_layout(
        title='Profit/Loss Distribution',
        xaxis_title='Profit/Loss (%)',
        yaxis_title='Number of Trades',
        bargap=0.1
    )
    
    fig.add_vline(x=0, line_dash='dash', line_color='red')
    
    # Add mean line
    mean_profit = backtest_results['profit_pct'].mean()
    fig.add_vline(x=mean_profit, line_color='green', 
                 annotation=dict(text=f"Mean: {mean_profit:.2f}%"))
    
    fig.show()
    
    # 3. Trade duration vs. profit scatter plot
    fig = px.scatter(
        backtest_results,
        x='bars_held',
        y='profit_pct',
        color='outcome',
        color_discrete_map={'WIN': 'green', 'LOSS': 'red', 'INCOMPLETE': 'gray'},
        hover_data=['timestamp', 'direction'],
        size='confidence' if 'confidence' in backtest_results.columns else None,
        size_max=15,
        opacity=0.7,
        title='Trade Duration vs. Profit'
    )
    
    fig.update_layout(
        xaxis_title='Bars Held',
        yaxis_title='Profit/Loss (%)',
    )
    
    fig.add_hline(y=0, line_dash='dash', line_color='black')
    
    # Add a trend line
    fig.update_layout(showlegend=True)
    
    fig.show()
    
    # 4. Profit over time (cumulative returns)
    backtest_results_sorted = backtest_results.sort_values('timestamp')
    backtest_results_sorted['cumulative_return'] = (1 + backtest_results_sorted['profit_pct']/100).cumprod() - 1
    backtest_results_sorted['cumulative_return_pct'] = backtest_results_sorted['cumulative_return'] * 100
    
    fig = px.line(
        backtest_results_sorted,
        x='timestamp',
        y='cumulative_return_pct',
        title='Cumulative Returns Over Time'
    )
    
    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Cumulative Return (%)',
    )
    
    # Add a zero line
    fig.add_hline(y=0, line_dash='dash', line_color='red')
    
    # Add markers for each trade
    fig.add_trace(
        go.Scatter(
            x=backtest_results_sorted['timestamp'],
            y=backtest_results_sorted['cumulative_return_pct'],
            mode='markers',
            marker=dict(
                size=8,
                color=['green' if x == 'WIN' else 'red' if x == 'LOSS' else 'gray' 
                      for x in backtest_results_sorted['outcome']]
            ),
            name='Trades'
        )
    )
    
    fig.show()
else:
    print("No backtest results available. Skipping performance metrics visualization.")

## 6. Feature Importance Tree Map

Let's create a tree map to visualize feature importance if we have a trained model.

In [ ]:
try:
    # Try to load the trained model
    model = RandomForestModel.load('../trained_model.pkl')
    print("Loaded saved model successfully")
    
    # Get feature importance
    feature_importance = model.get_feature_importance()
    
    # Create tree map
    fig = px.treemap(
        feature_importance.head(20),  # Top 20 features
        path=['Feature'],
        values='Importance',
        color='Importance',
        color_continuous_scale='Viridis',
        title='Feature Importance Tree Map'
    )
    
    fig.update_layout(
        height=600,
        width=900
    )
    
    fig.show()
except FileNotFoundError:
    print("Trained model not found. Skipping feature importance visualization.")

## 7. Calendar Heatmap

Let's create a calendar heatmap to visualize trading performance by day.

In [ ]:
if backtest_results is not None and len(backtest_results) > 20:
    # Ensure timestamp is datetime
    if not pd.api.types.is_datetime64_any_dtype(backtest_results['timestamp']):
        backtest_results['timestamp'] = pd.to_datetime(backtest_results['timestamp'])
    
    # Extract date components
    backtest_results['date'] = backtest_results['timestamp'].dt.date
    backtest_results['dayofweek'] = backtest_results['timestamp'].dt.dayofweek
    backtest_results['week'] = backtest_results['timestamp'].dt.isocalendar().week
    backtest_results['month'] = backtest_results['timestamp'].dt.month
    backtest_results['year'] = backtest_results['timestamp'].dt.year
    
    # Group by date
    daily_performance = backtest_results.groupby('date')['profit_pct'].sum().reset_index()
    daily_performance['dayofweek'] = pd.to_datetime(daily_performance['date']).dt.dayofweek
    daily_performance['week'] = pd.to_datetime(daily_performance['date']).dt.isocalendar().week
    daily_performance['month'] = pd.to_datetime(daily_performance['date']).dt.month
    daily_performance['month_name'] = pd.to_datetime(daily_performance['date']).dt.month_name()
    daily_performance['year'] = pd.to_datetime(daily_performance['date']).dt.year
    
    # Convert date to string
    daily_performance['date_str'] = daily_performance['date'].astype(str)
    
    # Create heatmap
    fig = px.density_heatmap(
        daily_performance,
        x='dayofweek',
        y='week',
        z='profit_pct',
        color_continuous_scale=[
            [0, 'red'],     # 0% of range is red
            [0.5, 'white'], # 50% of range is white
            [1, 'green']    # 100% of range is green
        ],
        color_continuous_midpoint=0,
        facet_row='month',
        facet_col='year',
        labels={
            'dayofweek': 'Day of Week',
            'week': 'Week',
            'profit_pct': 'Profit/Loss (%)',
            'month': 'Month',
            'year': 'Year'
        },
        title='Daily Trading Performance Calendar Heatmap',
        hover_data=['date_str', 'profit_pct']
    )
    
    # Update x-axis labels
    day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for annotation in fig.layout.annotations:
        if 'Day of Week=' in annotation.text:
            day_idx = int(annotation.text.split('=')[1])
            annotation.text = day_labels[day_idx]
    
    fig.update_layout(
        height=900,
        width=1200
    )
    
    fig.show()
else:
    print("Not enough backtest results available for calendar heatmap.")

## 8. Export Visualizations

Let's export some of our visualizations for use in reports or presentations.

In [ ]:
# Create a directory for exported visualizations
export_dir = '../visualizations'
os.makedirs(export_dir, exist_ok=True)

# Use matplotlib to create and save high-resolution images
if backtest_results is not None:
    # 1. Create and save equity curve
    backtest_results_sorted = backtest_results.sort_values('timestamp')
    if 'cumulative_return_pct' not in backtest_results_sorted.columns:
        backtest_results_sorted['cumulative_return'] = (1 + backtest_results_sorted['profit_pct']/100).cumprod() - 1
        backtest_results_sorted['cumulative_return_pct'] = backtest_results_sorted['cumulative_return'] * 100
    
    plt.figure(figsize=(12, 6), dpi=300)
    plt.plot(backtest_results_sorted['timestamp'], backtest_results_sorted['cumulative_return_pct'])
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    plt.title('Strategy Equity Curve')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return (%)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{export_dir}/equity_curve.png")
    plt.close()
    
    print(f"Saved equity curve to {export_dir}/equity_curve.png")
    
    # 2. Create and save win/loss pie chart
    win_count = len(backtest_results[backtest_results['outcome'] == 'WIN'])
    loss_count = len(backtest_results[backtest_results['outcome'] == 'LOSS'])
    incomplete_count = len(backtest_results[backtest_results['outcome'] == 'INCOMPLETE'])
    
    plt.figure(figsize=(8, 8), dpi=300)
    plt.pie([win_count, loss_count, incomplete_count], 
           labels=['Win', 'Loss', 'Incomplete'],
           colors=['green', 'red', 'gray'],
           autopct='%1.1f%%',
           startangle=90,
           wedgeprops=dict(width=0.5))
    plt.title('Trade Outcome Distribution')
    plt.tight_layout()
    plt.savefig(f"{export_dir}/outcome_distribution.png")
    plt.close()
    
    print(f"Saved outcome distribution to {export_dir}/outcome_distribution.png")
    
    # 3. Create and save profit/loss histogram
    plt.figure(figsize=(10, 6), dpi=300)
    plt.hist(backtest_results['profit_pct'], bins=20, alpha=0.7, color='blue')
    plt.axvline(x=0, color='r', linestyle='--', alpha=0.5)
    plt.axvline(x=backtest_results['profit_pct'].mean(), color='g', linestyle='-', alpha=0.7,
              label=f"Mean: {backtest_results['profit_pct'].mean():.2f}%")
    plt.title('Profit/Loss Distribution')
    plt.xlabel('Profit/Loss (%)')
    plt.ylabel('Number of Trades')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{export_dir}/profit_distribution.png")
    plt.close()
    
    print(f"Saved profit distribution to {export_dir}/profit_distribution.png")

# Check if we have a trained model
try:
    # Try to load the model if we don't already have it
    if 'model' not in locals() or model is None:
        model = RandomForestModel.load('../trained_model.pkl')
    
    # Create and save feature importance chart
    feature_importance = model.get_feature_importance()
    
    plt.figure(figsize=(12, 8), dpi=300)
    plt.barh(feature_importance['Feature'].head(15), feature_importance['Importance'].head(15))
    plt.title('Top 15 Feature Importance')
    plt.xlabel('Importance')
    plt.gca().invert_yaxis()  # To have the highest importance at the top
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{export_dir}/feature_importance.png")
    plt.close()
    
    print(f"Saved feature importance chart to {export_dir}/feature_importance.png")
except FileNotFoundError:
    print("Trained model not found. Skipping feature importance export.")

print(f"\nAll visualizations exported to {export_dir}/")

## Summary

In this notebook, we created a wide range of visualizations to better understand our trading data, model, and strategy performance:

1. Interactive price chart with technical indicators
2. Technical indicator dashboard showing price, RSI, MACD, and ATR
3. Detailed trade visualizations showing entry and exit points
4. Performance metrics dashboard with win rate, profit distribution, and cumulative returns
5. Feature importance tree map
6. Calendar heatmap for daily trading performance
7. Exported high-quality visualizations for reports and presentations

These visualizations help us gain deeper insights into our trading model's behavior and performance, making it easier to identify strengths, weaknesses, and potential improvements.